# Reweighting MC sample of $B \to D^{*-}\left(D^0 \to K^{-} \pi^+ \pi^+ \pi^-  \right)X$ Background

With `GBReweighter` in `hep_ml`

**INPUTS**
- `MC`: $B \to D^{*-}\left(D^0 \to K^{-} \pi^+ \pi^+ \pi^-  \right)X$ 
- `data`: LHCb data, with $_s$Weights to project in the $D^+ \to K^- \pi^+ \pi^-$ contribution and project out the other contributions.

**GOALS**
1. to learn the weights to apply to MC to align MC to data for the $D^0 \to K^- \pi^+ \pi^+ \pi^- $ decays by looking at the MC and $_s$Weighted LHCb data 
2. to apply the weights to the general MC.

We hope that this will reweight the general very inclusive MC sample for $B \to D^{*-} D^0 (X)$

In [1]:
import sys
sys.path.insert(0, '.')
from definition import (
    columns,
    data_names,
    column_ranges,
)

MC_name   = data_names['MC']
data_name = data_names['data']
reweighted_MC_name = 'GBReweighter_' + data_names['MC']

print("MC name: ", MC_name)
print("data name: ", data_name)
print("reweighted MC name: ", reweighted_MC_name)


MC name:  BTODstD0X_MC
data name:  BTODstD0X
reweighted MC name:  GBReweighter_BTODstD0X_MC


In [2]:
# python libraries
import numpy as np
import pandas as pd
import zfit
import timeit
from scipy.stats import ks_2samp
from sklearn.model_selection import train_test_split
from itertools import product

# hep_ml
from hep_ml.reweight import GBReweighter

# bd2dsttaunu
from bd2dsttaunu.locations import loc

# HEA library
from HEA.plot import plot_hist_auto, plot_hist, save_fig, plot_hist_var
from HEA import load_dataframe
from HEA.plot.tools import draw_vline
from HEA.definition import latex_params
from HEA.pandas_root import load_saved_root
from HEA.pandas_root import save_root
import HEA.BDT as bt
from HEA.tools.serial import dump_joblib, retrieve_joblib

Welcome to JupyROOT 6.22/06


## Read the dataframe

In [10]:
df = {}
df['MC'] = load_saved_root(MC_name + '_with_sWeights', folder_name=MC_name)
df['data'] = load_saved_root(data_name + '_with_sWeights', folder_name=data_name)

Loading /home/correiaa/bd2dsttaunu/output//root/BTODstD0X_MC/BTODstD0X_MC_with_sWeights.root
Loading /home/correiaa/bd2dsttaunu/output//root/BTODstD0X/BTODstD0X_with_sWeights.root


In [15]:
df['data'] = df['data'].dropna()
df['MC'] = df['MC'].dropna()

df['data'].reset_index(drop=True, inplace=True)
df['MC'].reset_index(drop=True, inplace=True)

## Prepare the dataframes

In [16]:
print(f"{len(columns)} available columns:")
for column in columns:
    print("- " + column)

13 available columns:
- m_Kpipipi
- q2_reco
- isolation_bdt
- tau_life_reco
- m_DstKpipi
- theta_X_reco
- theta_L_reco
- chi_reco
- costheta_X_reco
- costheta_L_reco
- coschi_reco
- tau_M
- B0_M


In [17]:
training_columns = [
    'costheta_X_reco',
    'costheta_L_reco',
    'chi_reco',
    'isolation_bdt',
    'q2_reco',
]

print(f"{len(training_columns)} columns used for training of the GBRWeighter:")
for training_column in training_columns:
    print("- " + training_column)

5 columns used for training of the GBRWeighter:
- costheta_X_reco
- costheta_L_reco
- chi_reco
- isolation_bdt
- q2_reco


We'll add the $_s$Weights to this list

### Split the dataframes in two

In [18]:
two_dfs = {
    1 : {},
    2 : {}
}

for type_df in df.keys():
    print(type_df)
    two_dfs[1][type_df] = df[type_df].sample(frac=0.5, random_state=20)
    two_dfs[2][type_df] = df[type_df].drop(two_dfs[1][type_df].sort_index().index,0).sample(frac=1., random_state=20)

# check
for type_df in df.keys():
    print(len(two_dfs[1][type_df]),len(two_dfs[2][type_df]))

MC
data
12768 12768
26615 26615


### Get the dataframes with only the training variables + the $_s$Weights

In [20]:
reweight_two_dfs = {
    1 : {},
    2 : {}
}
for num in reweight_two_dfs.keys():
    reweight_two_dfs[num]['MC'] = two_dfs[num]['MC'][training_columns]
    reweight_two_dfs[num]['data'] = two_dfs[num]['data'][training_columns + ['sWeight']]

In [21]:
for num in two_dfs.keys():
    for type_df in two_dfs[num].keys():
        assert (reweight_two_dfs[num]['MC'].columns == training_columns).all()
        assert (reweight_two_dfs[num]['data'].columns == training_columns + ['sWeight']).all()

In [22]:
### Divide into training and test samples

In [24]:
train_reweight_two_dfs = {
    1 : {},
    2 : {}
}
test_reweight_two_dfs = {
    1 : {},
    2 : {}
}

for num in two_dfs.keys():
    for type_df in two_dfs[num].keys():
        train_reweight_two_dfs[num][type_df], test_reweight_two_dfs[num][type_df] =\
            train_test_split(reweight_two_dfs[num][type_df], 
                             test_size=0.5,
                             random_state=20)

In [25]:
for type_df in two_dfs.keys():
    for num in two_dfs[type_df].keys():
        print(len(train_reweight_two_dfs[type_df][num]), len(test_reweight_two_dfs[type_df][num]))

6384 6384
13307 13308
6384 6384
13307 13308


## Hyperparameters tuning 

In [27]:
from HEA.tools import get_chi2_2samp

In [28]:
def compute_score(train_weights, test_weights):
    return ks_2samp(train_weights, test_weights).statistic

def compute_pvalue(train_weights, test_weights):
    return ks_2samp(train_weights, test_weights).pvalue

def compute_chi2(data_test_df, MC_test_df, data_test_weight, MC_test_weight, n_bins=15):
    columns = data_test_df.columns
    chi2 = 0
    for column in columns:
        data = data_test_df[column]
        original_MC = MC_test_df[column]
                
        chi2 += get_chi2_2samp(original_MC, data, 
                               n_bins=n_bins,
                               low=None, high=None,
                               weights1=MC_test_weight, 
                               weights2=data_test_weight)
    return chi2

def get_best_params(param_grid,
                    original_train_df, target_train_df, 
                    original_test_df, target_test_df,
                    target_train_weight=None, 
                    target_test_weight=None,
                    **fit_params):
    best_param_values = {}

    param_values_lists = product(*param_grid.values())
    param_names = param_grid.keys()
    
    best_score = np.inf
    
    scores = {}
    p_values = {}
    
    for param_values_list in param_values_lists:
        
        hyperparams = dict(zip(param_names, param_values_list))
        print("test of ")
        print(hyperparams)
        
        reweighter = GBReweighter(gb_args={'random_state': 20},
                                  **hyperparams)
        reweighter = reweighter.fit(
            original=original_train_df,
            target=target_train_df,
            target_weight=target_train_weight,
            **fit_params
        )
        
        MC_test_weight = reweighter.predict_weights(original_test_df)
        MC_train_weight = reweighter.predict_weights(original_train_df)
        
        scores[tuple(param_values_list)] = compute_chi2(
            data_test_df=target_test_df,
            MC_test_df=original_test_df,
            data_test_weight=target_test_weight,
            MC_test_weight=MC_test_weight,
        )
        
        p_values[tuple(param_values_list)] = compute_pvalue(MC_train_weight, MC_test_weight)
        
        
        if scores[tuple(param_values_list)] < best_score:
            best_score = scores[tuple(param_values_list)]
            best_param_values = tuple(param_values_list)
        
    return scores, p_values, best_param_values  


In [ ]:
param_grid = {
    "n_estimators": np.arange(10, 66, 5),
    'learning_rate': [0.08],
    "max_depth": [2]
}

scores = {}
best_param_values = {}
p_values = {}
for num in train_reweight_two_dfs.keys():
    scores[num], p_values[num], best_param_values[num] = get_best_params(
        param_grid,
        original_train_df=train_reweight_two_dfs[num]['MC'], 
        target_train_df=train_reweight_two_dfs[num]['data'].drop('sWeight', 1),
        original_test_df=test_reweight_two_dfs[num]['MC'],
        target_test_df=test_reweight_two_dfs[num]['data'].drop('sWeight', 1),
        target_train_weight=train_reweight_two_dfs[num]['data']['sWeight'],
        target_test_weight=test_reweight_two_dfs[num]['data']['sWeight'])